In [3]:
import pandas as pd

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

In [1]:
import os
from functools import reduce, partial
from utils import read_yaml
from pathlib import Path
import usuarios
import backup
import vpn_connection as vpn
import usuarios
import acidentes

root_dir = Path().resolve().parent

# Importa configurações do sistema
cfg = read_yaml(root_dir / 'config/config.yaml')
secrets = read_yaml(root_dir / 'config/secrets.yaml')
fatores_params = read_yaml(root_dir / 'config/fatores_risco_classificacao.yaml')
fatores_params_reshaped = acidentes.reshape_fatores_params(fatores_params['fatores_params'])

# Importa dados que populam as opções do formulário de inscrição
opcoes = usuarios.import_google_spreadsheet(cfg['FORM_INSC_OPCOES']['ID_GSHEET'])

# Importa listas de inscrições e cancelamentos e compila lista de usuários ativos
inscricoes = usuarios.import_google_spreadsheet(cfg['FORM_INSC']['ID_GSHEET'])
cancelamentos = usuarios.import_google_spreadsheet(cfg['FORM_CANCEL']['ID_GSHEET'])
usuarios = usuarios.compila_inscricoes(df_inscricao=inscricoes, df_cancelamento=cancelamentos)

# Realiza backup dos dados provenientes dos formulários de inscrição e cancelamento
backup_dir = str(root_dir / 'data/backup')

backup.backup_csv_new_file(opcoes, directory=backup_dir, filename='opcoes_form_insc.csv')
backup.backup_csv(inscricoes, backup_path=os.path.join(backup_dir, 'inscricoes.csv'))
backup.backup_csv(cancelamentos, backup_path=os.path.join(backup_dir, 'cancelamentos.csv'))
backup.backup_csv(usuarios, backup_path=os.path.join(backup_dir, 'usuarios.csv'))

# Tenta conectar à VPN
vpn.try_connection_forticlient_vpn(vpn_path=cfg['VNP_PATH'],
                                    user=secrets['VNP_USER'],
                                    password=secrets['VNP_PASSWORD'],
                                    url_test_connection=cfg['VPN_URL_TEST_CONNECTION'])

# Importa novas CATs
cats = acidentes.cat_extrair()

# Trata CATs
cat_atribui_fatores_risco_partial = partial(acidentes.cat_atribui_fatores_risco,
                                            fatores_params_reshaped=fatores_params_reshaped)

functions_list = [acidentes.cat_eliminar_ja_alertadas,
                    acidentes.cat_converter_datas,
                    acidentes.cat_formatar_horas,
                    acidentes.cat_formatar_strings,
                    acidentes.cat_cid_uppercase,
                    acidentes.cat_novas_colunas,
                    acidentes.cat_uorg_local_acidente,
                    acidentes.cat_formatar_datas,
                    acidentes.cat_identifica_recibo_raiz,
                    acidentes.cat_mantem_recibo_ultima_reabertura,
                    cat_atribui_fatores_risco_partial,
                    acidentes.cat_compila_fatores_risco,
                    acidentes.cat_atribui_consequencia,
                    acidentes.cat_inserir_descricoes
                    ]

cats_tratadas = reduce(lambda x, y: y(x), functions_list, cats)

Reading configuration file: C:\Users\joao.reis\Pywinauto recorder\config.ini
Window filtering mode: ignore_windows
Ignored windows: []


In [12]:
cats_tratadas.head(3)

,tpinsc,nrinsc,localtabgeral_tpinsc,localtabgeral_nrinsc,inporte,cnae_localtabgeral,razao_social,municipio_empregador,sguf_empregador,tpinsc_estab_local_acidente,nrinsc_estab_local_acidente,razao_social_estab_local_acidente,cnae_local_acidente,municipio_estab_local_acidente,sguf_estab_local_acidente,cpftrab,nistrab,matricula,nmtrab,dtadm,codcbo,nmcargo,sexo,grauinstr,racacor,dtnascto,codcateg,vrsalfx,tpacid,tpcat,dtacid,hracid,hrstrabantesacid,tplocal_acidente,dslocal_acidente,tplograd_local_acidente,dslograd_local_acidente,nr_lograd_local_acidente,complemento_local_acidente,bairro_local_acidente,cep_local_acidente,municipio_local_acidente,sguf_local_acidente,pais_local_acidente,codpostal_local_acidente,codagntcausador,codsitgeradora,codparteating,lateralidade,dsclesao,dsccomplesao,diagprovavel,codcid,obsatestado,indinternacao,indcatobito,dtobito,indafast,durtrat,dtatendimento,hratendimento,nmemit,ideoc,nroc,ufoc,iniciatcat,indretif,nrrecibo,procemi,meta_row_key,meta_nr_recibo,indcomunpolicia,nrRecCatOrig,idade_DTAcidente,DTEmissaoCAT,CDEmitenteCAT,NRCAT,codcidCategoria,uorg_local_acidente,recibo_raiz,codsitgeradora_fr,codagntcausador_fr,dsclesao_fr,codcid_fr,codcidCategoria_fr,CDFatorAmbiental,Consequencia,ds_tpacid,ds_tplocal_acidente,ds_tplograd_local_acidente,ds_municipio_local_acidente,ds_pais_local_acidente,ds_codagntcausador,ds_codsitgeradora,ds_codparteating,ds_lateralidade,ds_dsclesao,ds_codcid,ds_ideoc,ds_codcbo,ds_grauinstr,ds_racacor,ds_codcateg,ds_tpinsc,ds_localtabgeral_tpinsc,ds_cnae_localtabgeral,ds_municipio_empregador,ds_tpinsc_estab_local_acidente,ds_cnae_local_acidente,ds_municipio_estab_local_acidente,ds_CDEmitenteCAT,ds_iniciatcat,ds_tpcat,ds_indretif,ds_procemi,ds_inporte
0,1,01435328,1.0,01435328000284,NaN,1052000,COOPERATIVA REGIONAL DE COMERCIALIZACAO DO EXTREMO OESTE - COOPEROESTE,4217204,SC,1.0,01435328000284,COOPERATIVA REGIONAL DE COMERCIALIZACAO DO EXTREMO OESTE - COOPEROESTE,1052000,4217204,SC,09700087913,None,SECOOPEROE00000000000000000383,ELIEZER LUCIA LUNKES,07/01/2020,841505,OPERADOR(A) DE MáQ. DE LATIC.,F,8.0,6.0,19/11/1994,101.0,2400.70,1,1,12/08/2022,14:00,08:20,1,None,None,"Linha Bela das Flores-BR 163 SC, KM 76",S/N,None,None,89900000,4217204,SC,None,None,200008900,200008900,755070000,2,702010000,None,None,S601,None,N,N,NaN,S,3,12/08/2022,15:02,Gabriela de Mendona Rocha,1,27407,SC,1,1,None,3,202208150000000016654531203,1.1.0000000015805763531,N,None,28.0,15/08/2022,1,1.1.0000000015805763531,S60,023902000,1.1.0000000015805763531,NaN,NaN,NaN,NaN,NaN,[],[],Típico,Estabelecimento do empregador no Brasil,NaN,São Miguel do Oeste,NaN,"Impacto sofrido por pessoa, NIC","Impacto sofrido por pessoa, NIC",Dedo,Direita,"Corte, laceração, ferida contusa, punctura (ferida aberta)",Contusão de dedo(s) com lesão da unha,Conselho Regional de Medicina - CRM,Trabalhador de tratamento do leite e fabricação de laticínios e afins,Educação superior incompleta,Não informado,Empregado e Trabalhador Temporário,CNPJ,CNPJ,Fabricação de laticínios,São Miguel do Oeste,CNPJ,Fabricação de laticínios,São Miguel do Oeste,Empregador,Empregador,Inicial,Original,Aplicativo governamental - Web Geral,NaN
1,1,45232246,1.0,45232246001956,NaN,8610101,UNIMED DE RIBEIRAO PRETO COOPERATIVA DE TRABALHO MEDICO,3543402,SP,1.0,45232246001956,UNIMED DE RIBEIRAO PRETO COOPERATIVA DE TRABALHO MEDICO,8610101,3543402,SP,31270582844,12644131148,1-19-00000410,HELSO ANTONIO GASPARIN JUNIOR,12/06/2017,322205,TECNICO DE ENFERMAGEM II,M,7.0,1.0,09/12/1983,101.0,2441.23,1,1,11/08/2022,20:30,00:00,1,2º ANDAR,None,RUA AUXILIAR CHACARA OLHOS DAGUA,105,None,COUNTRY VILLAGE,14110000,3543402,SP,None,None,303010900,200080901,755070000,0,704020000,PERFURO,PERFURO,Z579,PERFURO,N,N,NaN,N,0,11/08/2022,20:35,SERGIO LUIZ WALTER DE ASSIS,1,38082,SP,1,1,None,1,202208150000000016654536587,1.1.0000000015805768948,N,None,39.0,15/08/2022,1,1.1.0000000015805768948,Z57,018917000,1.1.0000000015805768948,311,612,311,NaN,711,"[311, 612, 711]",[],Típico,Estabelecimento do